In [ ]:
%load_ext watermark


In [ ]:
from IPython.display import display
from matplotlib import ticker as mpl_ticker
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp
from tqdm import tqdm


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "wse-traits"
teeplot_subdir


## Prep Data


In [ ]:
dfxs_list = []

for data in tqdm([
    {
        "url": "https://osf.io/x6uvs/download",
        "initial conditions": "50/50",
        "genome model": "site-explicit",
    },
    {
        "url": "https://osf.io/bhk27/download",
        "initial conditions": "50/50",
        "genome model": "counter-based",
    },
    {
        "url": "https://osf.io/4mdws/download",
        "initial conditions": "de novo",
        "genome model": "site-explicit",
    },
    {
        "url": "https://osf.io/eacvu/download",
        "initial conditions": "de novo",
        "genome model": "counter-based",
    },
]):
    dfx = pd.read_parquet(data["url"])
    dfx["initial conditions"] = data["initial conditions"]
    dfx["genome model"] = data["genome model"]
    print("apn", dfx["population size"].min())
    print(dfx["population size"].unique())
    dfxs_list.append(dfx)


In [ ]:
dfxs = pd.concat(
    dfxs_list,
    ignore_index=True,
    sort=False,
)
display(dfxs.describe()), display(dfxs.head()), display(dfxs.tail());


In [ ]:
# filter to population sizes shared in all data
dfxs = dfxs[
    dfxs["population size"] != 190848000
].reset_index(drop=True)


In [ ]:
apn = dfxs["population size"].min()  # agents per node
apn


In [ ]:
dfxs["initial\nconditions"] = dfxs["initial conditions"]
dfxs["genome\nmodel"] = dfxs["genome model"]
dfxs["fix\nprob"] = dfxs["fixation probability"]


## initial conditions


In [ ]:
row = "population size"
for errorbar in ["sd", "se", "ci", None]:
    for what, data in dfxs.groupby("genome model"):
        with tp.teed(
            sns.relplot,
            data=data[
                data["genotype"] == "hypermutator"
            ],
            x="available beneficial mutations",
            y="fix\nprob",
            row=row,
            row_order=sorted(
                data[row].unique(), reverse=True
            ),
            hue="initial\nconditions",
            hue_order=["de novo", "50/50"],
            style="initial\nconditions",
            style_order=["de novo", "50/50"],
            aspect=8,
            errorbar=errorbar,
            # facet_kws=dict(margin_titles=True),
            height=0.65,
            kind="line",
            markers=True,
            palette="Set2_r",
            seed=1,
            teeplot_subdir=teeplot_subdir,
            teeplot_outattrs={
                "what": what.replace(" ", "-").replace("/", "-"),
            },
        ) as teed:
            teed.set_titles(
                col_template="",
                row_template="",
            )
            teed.set(
                ylim=(0, 1),
                xlim=(1, 16),
            )
            sns.move_legend(
                teed,
                "lower center",
                bbox_to_anchor=(0.45, 0.97),
                frameon=False,
                ncol=2,
                title=None,
                columnspacing=0.7,
            )
            for ax in teed.axes.flat:
                ax.axhline(0.5, ls=":", c="gray", lw=1)
                ax.set_yticks([0, 0.5, 1])
                ax.set_yticklabels(["", "0.5", ""])
                ax.set_ylabel("")

            teed.tight_layout()

            teed.figure.subplots_adjust(top=0.85)
            delta = 0.001  # Small height for the dummy axis
            pos = teed.axes.flat[0].get_position()
            dummy_ax = ax.figure.add_axes(
                [
                    pos.x0 + pos.width * 1.1,
                    pos.y0 - pos.height * data[row].nunique() * 1.3,
                    delta,
                    pos.height * data[row].nunique() * 1.7,
                ],
            )
            yvals = data[row]
            dummy_ax.set_ylim(yvals.min(), yvals.max())
            dummy_ax.yaxis.set_label_position("right")
            dummy_ax.set_ylabel(row)

            dummy_ax.set_yscale("log")
            dummy_ax.yaxis.set_ticks_position("right")
            dummy_ax.yaxis.set_major_locator(
                mpl_ticker.LogLocator(base=10),
            )
            formatter = mpl_ticker.LogFormatterMathtext(base=10)
            dummy_ax.yaxis.set_major_formatter(formatter)

            # Hide unwanted spines
            dummy_ax.get_xaxis().set_visible(False)

            teed.tight_layout()


## genome model


In [ ]:
row = "population size"
for errorbar in ["sd", "se", "ci", None]:
    for what, data in dfxs.groupby("initial conditions"):
        with tp.teed(
            sns.relplot,
            data=data[
                data["genotype"] == "hypermutator"
            ],
            x="available beneficial mutations",
            y="fix\nprob",
            row=row,
            row_order=sorted(
                data[row].unique(), reverse=True
            ),
            hue="genome\nmodel",
            hue_order=["site-explicit", "counter-based"],
            style="genome\nmodel",
            style_order=["site-explicit", "counter-based"],
            aspect=8,
            errorbar=errorbar,
            # facet_kws=dict(margin_titles=True),
            height=0.65,
            kind="line",
            markers=True,
            palette="Accent",
            seed=1,
            teeplot_subdir=teeplot_subdir,
            teeplot_outattrs={
                "what": what.replace(" ", "-").replace("/", "-"),
            },
        ) as teed:
            teed.set_titles(
                col_template="",
                row_template="",
            )
            teed.set(
                ylim=(0, 1),
                xlim=(1, 16),
            )
            sns.move_legend(
                teed,
                "lower center",
                bbox_to_anchor=(0.45, 0.97),
                frameon=False,
                title=None,
                ncol=2,
                columnspacing=0.7,
            )
            for ax in teed.axes.flat:
                ax.axhline(0.5, ls=":", c="gray", lw=1)
                ax.set_yticks([0, 0.5, 1])
                ax.set_yticklabels(["", "0.5", ""])
                ax.set_ylabel("")

            teed.tight_layout()

            teed.figure.subplots_adjust(top=0.85)
            delta = 0.001  # Small height for the dummy axis
            pos = teed.axes.flat[0].get_position()
            dummy_ax = ax.figure.add_axes(
                [
                    pos.x0 + pos.width * 1.1,
                    pos.y0 - pos.height * data[row].nunique() * 1.3,
                    delta,
                    pos.height * data[row].nunique() * 1.7,
                ],
            )
            yvals = data[row]
            dummy_ax.set_ylim(yvals.min(), yvals.max())
            dummy_ax.yaxis.set_label_position("right")
            dummy_ax.set_ylabel(row)

            dummy_ax.set_yscale("log")
            dummy_ax.yaxis.set_ticks_position("right")
            dummy_ax.yaxis.set_major_locator(
                mpl_ticker.LogLocator(base=10),
            )
            formatter = mpl_ticker.LogFormatterMathtext(base=10)
            dummy_ax.yaxis.set_major_formatter(formatter)

            # Hide unwanted spines
            dummy_ax.get_xaxis().set_visible(False)

            teed.tight_layout()
